<a href="https://colab.research.google.com/github/yunmokoo/SPY-Forecast_MARS_LSTM/blob/main/SPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests > make_outputrerquest.txt 2>&1
!pip install yfinance > make_outputyfinance.txt 2>&1
!pip install transformers > make_outputtransformers.txt 2>&1
!pip install xformers > make_outputxformers.txt 2>&1


In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz > wget_output.txt 2>&1
!tar -xzvf ta-lib-0.4.0-src.tar.gz > tar_output.txt 2>&1
%cd ta-lib
!./configure --prefix=/usr > config_output.txt 2>&1
!make > make_outputtalib.txt 2>&1
!make install > makeInstall_output.txt 2>&1
!pip install TA-Lib > pipInstall_output.txt 2>&1

/content/ta-lib


In [6]:
import requests
import json
import yfinance as yf
#import talib

import tensorflow as tf
import numpy as np
import pandas as pd
import torch
import matplotlib as plt

from transformers import *
from transformers import GenerationMixin
from transformers import TFGenerationMixin
from transformers import FlaxGenerationMixin

from sklearn.preprocessing import MinMaxScaler

import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import torch.optim as optim

from transformers import DistilBertTokenizer, DistilBertModel

From TA-lib...

1. MA                  All Moving Average
2. BBANDS              Bollinger Bands
3. MACD                Moving Average Convergence/Divergence
4. SMA                 Simple Moving Average
5. EMA                 Exponential Moving Average

In [14]:

data_SPY = yf.download("SPY", start = "2001-01-01", end = "2023-01-01")


print(data_SPY)

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2001-01-02  132.000000  132.156250  127.562500  128.812500   84.662971   
2001-01-03  128.312500  136.000000  127.656250  135.000000   88.729759   
2001-01-04  134.937500  135.468750  133.000000  133.546875   87.774666   
2001-01-05  133.468750  133.625000  129.187500  129.187500   84.909431   
2001-01-08  129.875000  130.187500  127.687500  130.187500   85.566734   
...                ...         ...         ...         ...         ...   
2022-12-23  379.649994  383.059998  378.029999  382.910004  380.042480   
2022-12-27  382.790009  383.149994  379.649994  381.399994  378.543793   
2022-12-28  381.329987  383.390015  376.420013  376.660004  373.839294   
2022-12-29  379.630005  384.350006  379.079987  383.440002  380.568481   
2022-12-30  380.640015  382.579987  378.429

In [4]:
df = pd.DataFrame(data_SPY)
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2001-01-02,132.000000,132.156250,127.562500,128.812500,84.662979,8737500
2001-01-03,128.312500,136.000000,127.656250,135.000000,88.729744,19431600
2001-01-04,134.937500,135.468750,133.000000,133.546875,87.774696,9219000
2001-01-05,133.468750,133.625000,129.187500,129.187500,84.909470,12911400
2001-01-08,129.875000,130.187500,127.687500,130.187500,85.566704,6625300
...,...,...,...,...,...,...
2022-12-23,379.649994,383.059998,378.029999,382.910004,380.042480,59857300
2022-12-27,382.790009,383.149994,379.649994,381.399994,378.543793,51638200
2022-12-28,381.329987,383.390015,376.420013,376.660004,373.839264,70911500


# Transformer


In [5]:
scaler = MinMaxScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df_normalized

,Open,High,Low,Close,Adj Close,Volume
0,0.155737,0.151608,0.147845,0.148199,0.079748,0.006263
1,0.146771,0.160983,0.148074,0.163306,0.089547,0.018587
2,0.162880,0.159688,0.161140,0.159758,0.087246,0.006818
3,0.159308,0.155190,0.151818,0.149115,0.080342,0.011073
4,0.150570,0.146806,0.148150,0.151556,0.081925,0.003829
...,...,...,...,...,...,...
5530,0.757896,0.763598,0.760294,0.768555,0.791514,0.065175
5531,0.765531,0.763818,0.764256,0.764868,0.787902,0.055703
5532,0.761981,0.764403,0.756358,0.753296,0.776566,0.077915
5533,0.757848,0.766745,0.762862,0.769849,0.792781,0.073373


In [10]:
import torch
import torch.nn as nn
from transformers import DistilBertTokenizer, DistilBertModel

class SimpleTransformerModel(nn.Module):
    def __init__(self, num_classes):
        super(SimpleTransformerModel, self).__init__()
        self.tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
        self.encoder = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.fc = nn.Linear(self.encoder.config.hidden_size, num_classes)

    def forward(self, input_text):
        input_ids = self.tokenizer(input_text, return_tensors='pt', padding=True, truncation=True)['input_ids']
        outputs = self.encoder(input_ids)
        last_hidden_state = outputs.last_hidden_state
        cls_token = last_hidden_state[:, 0, :]
        logits = self.fc(cls_token)
        return logits

# Assuming you have prepared your data as train_loader and have defined num_classes.

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleTransformerModel(num_classes)
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()

# Training loop
num_epochs = 5  # Change this based on the number of epochs you want to run
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch_idx, batch_data in enumerate(train_loader):
        inputs, targets = batch_data
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_loss:.4f}")


loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_tok

ValueError: ignored